In [125]:
import pandas as pd
import numpy as np
import yfinance as yf
import math

# Input for number of stocks
while True:
    try:
        num_stocks = int(input("Please enter the number of stocks you want to input: "))
        if num_stocks <= 0:
            print("The number of stocks must be a positive integer. Please try again.")
        else:
            break
    except ValueError:
        print("Invalid input. Please enter a valid integer.")

stock_symbols = []
weights = []

# Get stock symbols and weights
for i in range(num_stocks):
    while True:
        stock_symbol = input(f"Enter stock symbol {i + 1} (e.g., AAPL): ")
        if stock_symbol.isalpha():  
            stock_symbols.append(stock_symbol.upper())
            break
        else:
            print("Invalid stock symbol. Please enter only letters.")
    
    while True:
        try:
            weight = float(input(f"Enter weight for {stock_symbols[-1]}: "))
            if weight <= 0:
                print("Weight must be a positive number. Please try again.")
            else:
                weights.append(weight)
                break
        except ValueError:
            print("Invalid input. Please enter a valid float number.")

print("\nStock symbols and weights collected:")
for symbol, weight in zip(stock_symbols, weights):
    print(f"Symbol: {symbol}, Weight: {weight}")

# Input for market reference
market_reference = input("Please input the market you would like to use for beta calculation (e.g., ^gspc): ")
stock_symbols.append(market_reference)


# Fetch data using yfinance
Yf_ticker_input = " ".join(stock_symbols).lower()
Stock = yf.Tickers(Yf_ticker_input)
df = Stock.history(period="max")

# Calculate returns
Closing_Prices = df["Close"]
log_returns = np.log(Closing_Prices / Closing_Prices.shift(1)).dropna()

# Beta calculation
def beta_calc(df):
    beta = []
    market_returns = df.iloc[:, -1]  # Market returns is the last column

    for stock in df.columns[:-1]:
        stock_returns = df[stock]
        covariance = np.cov(stock_returns, market_returns)[0, 1]
        beta.append(covariance / np.var(market_returns))
    
    return pd.Series(beta, index=df.columns[:-1], name="Beta")

# Weight calculation
def beta_weighted(list_of_weights):
    total = sum(list_of_weights)
    weights = [weight / total for weight in list_of_weights]
    return pd.Series(weights, index=range(len(list_of_weights)), name="beta_weight")



# Calculate betas and weights 
betas = beta_calc(log_returns)
weights = beta_weighted(weights)  # Pass actual weights here

# Ensure indices match
weights.index = betas.index

# Calculate final beta weighting
Final_beta_weighting = betas.dot(weights)
print("Your Portfolio has a Beta of: {}".format(Final_beta_weighting.round(3)))







Stock symbols and weights collected:
Symbol: AAPL, Weight: 3.0
Symbol: SOXL, Weight: 21.0


[*********************100%%**********************]  3 of 3 completed

Your Portfolio has a Beta of: 3.701


: 